# 02 RAG | 01 Chat Completion | 04 Create Image Completion

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))


## Step 1: Create OpenAIClient

The `AzureOpenAIClient` from the Azure.AI.OpenAI NuGet package offers a unified interface to create various specialized clients, each designed to handle specific tasks.
In this notebook the `ImageClient` is used to create a photorealistic image based on a text prompt.


In [1]:
#r "nuget: Azure.AI.OpenAI, 2.0.0-beta.2"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Images;
using DotNetEnv;
using System.IO;
using System.Text.Json; 
using System.ClientModel;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
string imageDeploymentName = Environment.GetEnvironmentVariable("WS_IMAGE_DEPLOYMENTNAME") ?? "WS_IMAGE_DEPLOYMENTNAME not found";
string assetsFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);
ImageClient imageClient = azureOpenAIClient.GetImageClient(imageDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"ImageClient created...");

Installed Packages Azure.AI.OpenAI, 2.0.0-beta.2 DotNetEnv, 2.5.0

AzureOpenAI Client created...
ImageClient created...


## Step 2: Create Images

The following cell, demonstrate a basic interaction using the `imageClient` instance. In the sample 3 images are created providing the following description:

- *"Create a photo realistic image of an US american football team winning the Super Bowl. Show them celebrating as team in a stadium with confetti falling from the sky"*
- *"Create an sketch style image of an US american football team winning the Super Bowl in a barely sketched out stadium. Show the team celebrating as team in a stadium with confetti falling from the sky"*
- *"Create an oilpainting image of an US american football team winning the Super Bowl in a historic stadium. Show the team celebrating as team in a stadium with confetti falling from the sky"*

The `imageGenerationOptions' used in the code cell allows more fine granular control how the image is created and in which format it is returned to the caller.

In [3]:
using System.Net.Http;

//Define Image Generation Options
ImageGenerationOptions imageGenerationOptions = new ImageGenerationOptions() {
    Quality = GeneratedImageQuality.Standard,
    ResponseFormat = GeneratedImageFormat.Bytes, 
    Size = GeneratedImageSize.W1024xH1024,
    Style = GeneratedImageStyle.Natural
};

//Generate Photorealistic Image
string imagePrompt = @"
    Create a photo realistic image of an US american football team winning the Super Bowl in a futuristic stadium. 
    Show the team celebrating as team in a stadium with confetti falling from the sky
";
string imageFileName = Path.Join(assetsFolder, "Image", "SuperBowl_Photorealistic.png");

ClientResult<GeneratedImage> result = await imageClient.GenerateImageAsync(
    imagePrompt, 
    imageGenerationOptions
);
File.WriteAllBytes(imageFileName, result.Value.ImageBytes.ToArray());
Console.WriteLine($"Image generated and saved to {imageFileName}...");         

//Generate Sketch Style Image
imagePrompt = @"
    Create an sketch style image of an US american football team winning the Super Bowl in a barely sketched out stadium. 
    Show the team celebrating as team in a stadium with confetti falling from the sky
";
imageFileName = Path.Join(assetsFolder, "Image", "SuperBowl_Sketch.png");

result = await imageClient.GenerateImageAsync(
    imagePrompt, 
    imageGenerationOptions
);
File.WriteAllBytes(imageFileName, result.Value.ImageBytes.ToArray());
Console.WriteLine($"Image generated and saved to {imageFileName}...");         


//Generate Artist Oilpainting Style Image
imagePrompt = @"
    Create an oilpainting image of an US american football team winning the Super Bowl in a historic stadium. 
    Show the team celebrating as team in a stadium with confetti falling from the sky
";
imageFileName = Path.Join(assetsFolder, "Image", "SuperBowl_Oilpainting.png");

result = await imageClient.GenerateImageAsync(
    imagePrompt, 
    imageGenerationOptions
);
File.WriteAllBytes(imageFileName, result.Value.ImageBytes.ToArray());
Console.WriteLine($"Image generated and saved to {imageFileName}...");         

Image generated and saved to ../../assets\Image\SuperBowl_Photorealistic.png...
Image generated and saved to ../../assets\Image\SuperBowl_Sketch.png...
Image generated and saved to ../../assets\Image\SuperBowl_Oilpainting.png...


The following images have been created in a previous execution. Images created using the same prompts will most likely look different due to the unpredicted nature of LLMs.

![PhotoRealisticImage](../../assets/Image/SuperBowl_Photorealistic.png) 
![SketchImage](../../assets/Image/SuperBowl_Sketch.png)
![OilpaintingImage](../../assets/Image/SuperBowl_Oilpainting.png)